<a href="https://colab.research.google.com/github/sergiodevelop78/pruebasTensorflow/blob/master/TensorFlow_Para_entrenar_Sacos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!unzip /content/drive/My\ Drive/models.zip -d /content/
!rm -rf /content/para_entrenar
!tar -xzf /content/drive/My\ Drive/para_entrenar.tar.gz --directory  /content/
!ls -lh /content/

total 12K
drwx------  3 root root 4.0K Apr  8 00:59 drive
drwxr-xr-x 10 1000 1000 4.0K Apr  6 23:39 para_entrenar
drwxr-xr-x  1 root root 4.0K Apr  3 16:24 sample_data


In [3]:
!ls -lh /content/para_entrenar/

total 172K
drwxr-xr-x  2 1000 1000 4.0K Apr  6 23:33 configuracion
drwxr-xr-x  2 1000 1000 128K Apr  6 23:38 img_entrenamiento
drwxr-xr-x  2 1000 1000  20K Apr  6 23:38 img_test
drwxr-xr-x  2 1000 1000 4.0K Apr  6 23:33 modelo
drwxr-xr-x 26 1000 1000 4.0K Apr  6 23:33 object_detection
drwxr-xr-x 63 1000 1000 4.0K Apr  6 23:33 research
drwxr-xr-x  2 1000 1000 4.0K Apr  6 23:34 TFRecords
drwxr-xr-x  2 1000 1000 4.0K Apr  6 23:39 train


In [0]:
%tensorflow_version 1.x
!pip install Cython
!pip install contextlib2 pillow lxml matplotlib PyDrive
!pip install pycocotools
!apt install protobuf-compiler python-pil python-lxml python-tk


In [5]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
%cd /content/para_entrenar/research
!protoc object_detection/protos/*.proto --python_out=.
!export PYTHONPATH=$PYTHONPATH:`pwd`/slim
!python object_detection/builders/model_builder_test.py

%cd /content/para_entrenar/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH']+=':/content/para_entrenar/research/:/content/para_entrenar/research/slim/'

!python object_detection/builders/model_builder_test.py

In [25]:
%cd /content/para_entrenar/
!export PYTHONWARNINGS="ignore"
!python3 -W ignore::DeprecationWarning object_detection/model_main.py --pipeline_config_path=modelo/faster_rcnn_resnet101_coco.config \
-num_train_steps=2000 --model_dir=train/ \
 --alsologtostderr

/content/para_entrenar
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0408 01:40:15.681247 140068195936128 module_wrapper.py:139] From /content/para_entrenar/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0408 01:40:15.683960 140068195936128 module_wrapper.py:139] From /content/para_entrenar/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0408 01:40:15.684110 140068195936128 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0408 01:40:15.684224 140068195936128 module_wrapp

In [26]:
!ls -lh /content/para_entrenar/train/


total 892M
-rw-r--r-- 1 root root  130 Apr  8 01:48 checkpoint
-rw-r--r-- 1 root root  27M Apr  8 01:49 events.out.tfevents.1586310028.24de716d10bf
-rw-r--r-- 1 root root  14M Apr  8 01:40 graph.pbtxt
-rw-r--r-- 1 root root 419M Apr  8 01:40 model.ckpt-0.data-00000-of-00001
-rw-r--r-- 1 root root  40K Apr  8 01:40 model.ckpt-0.index
-rw-r--r-- 1 root root 7.4M Apr  8 01:40 model.ckpt-0.meta
-rw-r--r-- 1 root root 419M Apr  8 01:48 model.ckpt-2000.data-00000-of-00001
-rw-r--r-- 1 root root  40K Apr  8 01:48 model.ckpt-2000.index
-rw-r--r-- 1 root root 7.4M Apr  8 01:48 model.ckpt-2000.meta


In [32]:
!rm -rf /content/para_entrenar/inference_graph
!mkdir /content/para_entrenar/inference_graph
%cd /content/para_entrenar/
#!ls -lh

!python3  object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path modelo/faster_rcnn_resnet101_coco.config \
--trained_checkpoint_prefix train/model.ckpt-2000 \
--output_directory inference_graph/


/content/para_entrenar
251 ops no flops stats due to incomplete shapes.
Parsing Inputs...
Incomplete shape.

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              0
-min_occurrence             0
-step                       -1
-order_by                   name
-account_type_regexes       _trainable_variables
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     params
-output                     stdout:

==================Model Analysis Report======================
Incomplete shape.

Doc:
scope: The nodes in the model graph are organized by their

In [33]:
%cd /content/para_entrenar/
!rm -rf inference_graph.tar.gz

!tar -zcvf inference_graph.tar.gz inference_graph/

!mv inference_graph.tar.gz /content/drive/My\ Drive


/content/para_entrenar
inference_graph/
inference_graph/model.ckpt.data-00000-of-00001
inference_graph/checkpoint
inference_graph/model.ckpt.index
inference_graph/model.ckpt.meta
inference_graph/frozen_inference_graph.pb
inference_graph/pipeline.config
inference_graph/saved_model/
inference_graph/saved_model/saved_model.pb
inference_graph/saved_model/variables/
